In [1]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain


llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print("OpenAI API 说答案是:", answer)

python_answer = 352 * 493
print("Python 说答案是:", python_answer)

OpenAI API 说答案是: 

352 x 493 = 174,336
Python 说答案是: 173536


In [11]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print(answer)



print(352 * 493)


In [13]:
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer_code)
print(result)

173536



In [15]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)



> Entering new LLMMathChain chain...
请计算一下352乘以493是多少?
```python
print(352 * 493)
```

Answer: 173536

> Finished chain.
Answer: 173536



In [44]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，如果里面没有相关信息的话就说 "找不到"
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)
requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "今天上海的天气怎么样？"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}
result=requests_chain(inputs)
print(result)
print(result['output'])

{'query': '今天上海的天气怎么样？', 'url': 'https://www.google.com/search?q=今天上海的天气怎么样？', 'output': '小雨; 10℃～15℃; 东北风 风力4-5级'}
小雨; 10℃～15℃; 东北风 风力4-5级


In [37]:
import re

def parse_weather_info(weather_info)->dict:
    # 将天气信息拆分成不同部分
    parts = weather_info.split('; ')

    # 解析天气
    weather = parts[0].strip()

    # 解析温度范围，并提取最小和最大温度
    temperature_range = parts[1].strip().replace('℃', '').split('～')
    temperature_min = int(temperature_range[0])
    temperature_max = int(temperature_range[1])

    # 解析风向和风力
    wind_parts = parts[2].split(' ')
    wind_direction = wind_parts[0].strip()
    wind_force = wind_parts[1].strip()
    weather_dict = {
        'weather': weather,
        'temperature': {
            'min': temperature_min,
            'max': temperature_max,
        },
        'wind': {
            'direction': wind_direction,
            'level': wind_force
        }
    }
    return weather_dict

weather_dict = parse_weather_info(result['output'])
print(weather_dict)

{'weather': '小雨', 'temperature': {'min': 10, 'max': 15}, 'wind': {'direction': '东北风', 'level': '风力4-5级'}}


In [45]:
from langchain.chains import TransformChain, SequentialChain

def transform_func(inputs: dict) -> dict:
    text = inputs["output"]
    return {"weather_info" : parse_weather_info(text)}

transformation_chain = TransformChain(input_variables=["output"], 
                                      output_variables=["weather_info"], transform=transform_func)

final_chain = SequentialChain(chains=[requests_chain, transformation_chain], 
                              input_variables=["query", "url"], output_variables=["weather_info"])
final_result = final_chain.run(inputs)

In [42]:
print(final_result)

{'weather': '小雨', 'temperature': {'min': 10, 'max': 15}, 'wind': {'direction': '东北风', 'level': '风力4-5级'}}


In [61]:
from langchain.memory import ConversationBufferWindowMemory

template = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
llm_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    memory=memory,
)
llm_chain.predict(human_input="你是谁？")

' 我是一个中国厨师，我可以帮助你做菜。我会根据你的口味和特殊要求，精心烹饪出独特美味的中国菜肴。'

In [62]:
llm_chain.predict(human_input="鱼香肉丝怎么做？")
llm_chain.predict(human_input="那宫保鸡丁呢？")
llm_chain.predict(human_input="我问你的第一句话是什么？")

' 你是谁？'

In [63]:
llm_chain.predict(human_input="我问你的第一句话是什么？")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


' 你问我的第一句话是“鱼香肉丝怎么做？”'

In [66]:
memory.load_memory_variables({})

{'chat_history': 'Human: 那宫保鸡丁呢？\nAI:  宫保鸡丁是一道经典的中国家常菜，需要准备鸡肉、花生米、干辣椒、葱、姜、蒜、料酒、盐、糖、胡椒粉、鸡精和醋。将鸡肉切成小块，放入盐水中浸泡，把其他食材切成小块，将花生米放入油锅中炸，再加入鸡肉和其他食材，炒至入味即可。\nHuman: 我问你的第一句话是什么？\nAI:  你是谁？\nHuman: 我问你的第一句话是什么？\nAI:  你问我的第一句话是“鱼香肉丝怎么做？”'}

In [82]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=OpenAI())

prompt_template = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

{history}
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=memory,
    prompt=prompt,
    verbose=True
)
conversation_with_summary.predict(input="你好")



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


Human: 你好
AI:

> Finished chain.


' 你好，我可以帮你做菜。我会根据你的口味和喜好，结合当地的食材，制作出美味可口的菜肴。我会尽力做出最好的菜肴，让你满意。'

In [83]:
conversation_with_summary.predict(input="鱼香肉丝怎么做？")



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


The human greeted the AI and the AI responded that it can help cook by combining local ingredients and tailor the meal to the human's tastes and preferences. It promised to make the best dishes possible to the human's satisfaction.
Human: 鱼香肉丝怎么做？
AI:

> Finished chain.


' 鱼香肉丝是一道经典的家常菜，需要准备肉丝、葱姜蒜、鱼香调料、豆瓣酱、醋、糖、盐等调料，先将肉丝用盐、料酒、胡椒粉腌制，然后炒锅里放入葱姜蒜爆香，加入肉丝翻炒，加入鱼香调料、豆瓣酱、醋、糖等调料，最后放入少许水煮熟即可。'

In [85]:
memory.load_memory_variables({})

{'history': ''}

In [86]:
conversation_with_summary.predict(input="那蚝油牛肉呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


The human greeted the AI and the AI responded that it can help cook by combining local ingredients and tailor the meal to the human's tastes and preferences. It promised to make the best dishes possible to the human's satisfaction. When asked how to make 鱼香肉丝, the AI responded that it requires the preparation of meat slices, scallion, ginger, garlic, fish sauce, doubanjiang, vinegar, sugar and salt. The meat slices should be marinated with salt, cooking wine and pepper, then stir-fried with scallion, ginger and garlic. The fish sauce, doubanjiang, vinegar, sugar and salt should be added in, with some water added to cook the dish.
Human: 那蚝油牛肉呢？
AI:

> Finished chain.


' 蚝油牛肉需要准备牛肉、蚝油、葱、姜、蒜、料酒、盐、糖、醋、淀粉和水。牛肉应先用盐、料酒和胡椒粉腌制，然后和葱、姜、蒜一起爆炒，再加入蚝油、糖、盐、醋和水，最后加入淀粉勾芡即可。'